In [34]:
import os
import os.path
import shutil
import tensorflow as tf
LOGDIR = "/tmp/mnist_tutorial/"
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=LOGDIR + "data", one_hot=True)

Extracting /tmp/mnist_tutorial/data/train-images-idx3-ubyte.gz
Extracting /tmp/mnist_tutorial/data/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist_tutorial/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist_tutorial/data/t10k-labels-idx1-ubyte.gz


In [35]:
def conv_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.zeros([5, 5, channels_in, channels_out]))
    b = tf.Variable(tf.zeros([channels_out]))
    # conv layer
    x = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding='SAME')
    # activation
    x = tf.nn.relu(x + b)
    return x

In [36]:
def fc_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.truncated_normal([channels_in, channels_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[channels_out]), name="B")
    x = tf.matmul(input, w) + b
    return x

In [39]:
def mnist_model(learning_rate, use_two_fc, use_two_conv, hparam):
    tf.reset_default_graph()
    sess = tf.Session()
    
    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    y = tf.placeholder(tf.float32, shape=[None, 10], name="y")
    
    # Create the network
    conv1 = conv_layer(x_image, 1, 32)
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv2 = conv_layer(pool1, 32, 64)
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    flatten = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    fc1 = fc_layer(flatten, 7 * 7 * 64, 1028)
    logits = fc_layer(fc1,  7*7*64, 10)
    
    # Loss
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
    )
    
    # Training setup
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
    # compute the accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # Actual training
    sess.run(tf.global_variables_initializer())
    
    # Train for 200 steps
    for i in range(200):
        batch = mnist.train.next_batch(100)
        
        # Ocassionally report accuracy
        if i % 500:
            [train_accuracy] = sess.run([accuracy], feed_dict={x: batch[:1024], y: batch[:1024]})
            print('Step %d, training accuracy %g' % (i, train_accuracy))
            
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

In [40]:
learning_rate=1E-3
mnist_model(learning_rate, True, True, True)

ValueError: Dimensions must be equal, but are 1028 and 1024 for 'MatMul_1' (op: 'MatMul') with input shapes: [?,1028], [1024,10].